In [1]:
! pip install konlpy

     |████████████████████████████████| 19.4 MB 104 kB/s 
     |████████████████████████████████| 448 kB 46.9 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Attention 신경망 구현 및 학습

In [3]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [4]:
EPOCHS = 200
NUM_WORDS = 5000

## Encoder

In [5]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        '''
        return_sequence=True로하는 이유
        : Attention mechanism을 사용할 때 우리가 key와 value는 Encoder에서 나오는 모든 Hidden state 부분을 사용해야 하므로
        '''
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        # embedding 결과 dimension : (batch_size, sequence 길이, embedding feature 수(64))

        H, h, c = self.lstm(x)
        # H : (batch_size, sequence 길이, LSTM unit 수)
        # h : (batch size, LSTM unit 수)
        # c : (batch size, LSTM unit 수)
        return H, h, c

## Decoder

In [11]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.attention = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        y_, s0, c0, H = inputs
        # y_ : shifted labels로 맨 마지막을 제외한 데이터들 (batch size, sequence 길이)
        # s0 : 이전 step의 hidden state (batch size, LSTM Unit 개수)
        # c0 : 이전 step의 cell state (batch size, LSTM Unit 개수)
        # H : Encoder 단의 모든 hidden state를 모은 것 (batch size, sequence길이, LSTM Unit 개수)
        
        y_ = self.emb(y_)
        # embedding 결과 dimension : (batch_size, sequence 길이, embedding feature 수(64))

        S, h, c = self.lstm(y_, initial_state=[s0, c0])
        # S : (batch_size, sequence 길이, LSTM unit 수)
        # h : (batch size, LSTM unit 수)
        # c : (batch size, LSTM unit 수)

        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        # S : (batch_size, sequence 길이, LSTM unit 수)

        A = self.attention([S_,H])
        # A : (batch_size, sequence 길이, LSTM unit 수)
        
        y = tf.concat([S, A], axis=-1)

        return self.dense(y), h, c

## Seq2seq

In [12]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [13]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:] # sos 미포함 eos 포함
    shifted_labels = labels[:, :-1] # sos 포함 eos 미포함
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [9]:
dataset_file = '/content/drive/MyDrive/패스트캠퍼스/Part4) 딥러닝 3 STEP의 기초/dataset/chatbot_data.csv'  # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines] # morphs : 형태소 분석

questions = seq[::2] # 홀수행
answers = ['\t ' + lines for lines in seq[1::2]] # 짝수행
# \t : sos, \n : eos

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [14]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [15]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 3.5171024799346924, Accuracy: 83.14144897460938
Epoch 2, Loss: 0.7384456396102905, Accuracy: 90.5349349975586
Epoch 3, Loss: 0.5959764122962952, Accuracy: 90.62891387939453
Epoch 4, Loss: 0.5666077136993408, Accuracy: 91.15757751464844
Epoch 5, Loss: 0.5485181212425232, Accuracy: 91.14191436767578
Epoch 6, Loss: 0.5396406054496765, Accuracy: 91.1536636352539
Epoch 7, Loss: 0.5366899967193604, Accuracy: 91.16541290283203
Epoch 8, Loss: 0.5281073451042175, Accuracy: 91.29072570800781
Epoch 9, Loss: 0.5223872661590576, Accuracy: 91.23981475830078
Epoch 10, Loss: 0.5095158815383911, Accuracy: 91.28681182861328
Epoch 11, Loss: 0.500338077545166, Accuracy: 91.3572998046875
Epoch 12, Loss: 0.47368568181991577, Accuracy: 91.502197265625
Epoch 13, Loss: 0.46952730417251587, Accuracy: 91.9525375366211
Epoch 14, Loss: 0.45724916458129883, Accuracy: 92.10917663574219
Epoch 15, Loss: 0.4508518576622009, Accuracy: 92.26973724365234
Epoch 16, Loss: 0.44539695978164673, Accuracy: 92.359

## 테스트 루프

In [16]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['아메리카노 나오면 진동 벨 로 알려 드리겠습니다 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['네 저기 서 직접 가져오시면 됩니다 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 카드 카드 카드 드릴게요 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['아니요 치즈케이크 는 지금 없어요 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 이 쪽 에 있습니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['아메리카노 가격 과 아이스 벨 로 주문 한 드릴 까요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['4100원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['10분 정도 걸려요 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가실 건가 요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['네 유효 에 담아 개 드리겠습니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['네 자몽 에 이드 없어요 \n']
_
q:  